# Module 11: Mapping

## Overview

Mapping short reads against a reference genome is typically the first step to analyze such next-generation sequencing data, and it should be as accurate as possible. Because of the high number of reads to handle, numerous sophisticated algorithms have been developped to tackle this problem and many mapping tools exist now.

### Selecting a reference sequence for mapping

A single reference genome is designated to represent a species for comparative analysis. A complete reference genome should be of high quality annotation and meets the highest level of experimental support for structural and functional annotation. 

>**Nota**: Note: It is important to have in mind that although a single arbitrary reference genome is a frequently used approach in microbial genomics, the choice of a reference may represent a source of errors that may affect subsequent analyses such as the detection of single nucleotide polymorphisms (SNPs) and phylogenetic inference.

*Further reading*: 

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3375638/

https://academic.oup.com/nar/article/42/D1/D553/1066302 

https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1008678 

### Install condacolab

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

### Install software

In [ ]:
# Install bwa
!conda install -c bioconda bwa

In [ ]:
# Install samtools
!conda config --add channels bioconda
!conda config --add channels conda-forge
!conda install -c bioconda samtools

### Download data

In [ ]:
!wget https://zenodo.org/records/14231070/files/Module_11.tar.gz

### Extract the .tar.gz file 

In [ ]:
!tar xvf Module_11.tar.gz

## Part 1: Assessing files 

Navigate to this folder and explore its contents. You should see the compressed FASTQ paired end read files:


- ERR2667737_1.fastq.gz

- ERR2667737_2.fastq.gz

In this module we will analyze sequences with run accession [ERR2667737](https://www.ebi.ac.uk/ena/browser/view/ERR2667737)  from the project [PRJEB3084](https://www.ebi.ac.uk/ena/browser/view/PRJEB3084).

Some important data about the sample:

- Country of origin: Argentina
- Organism: *Streptococcus pneumoniae*
- Instrument Platform: ILLUMINA
- Instrument Model: Illumina HiSeq X Ten
- Read Count: 2148156
- Base Count: 324371556
- Center Name: Wellcome Sanger Institute; SC
- Library Layaout: PAIRED
- Library strategy: WGS

And also two FASTA reference sequence files:

- Reference_sequence_GPSC1.fa

- Reference_sequence_GPSC33.fa

We will be aligning the paired end reads to the two reference sequences in turn. The reference sequences represent the GPSC33 and GPSC1 lineages of *Streptococcus pneumoniae*, and we will use the alignment results to determine which lineage the reads belong to. 

Further reading: https://pubmed.ncbi.nlm.nih.gov/31003929/

As a quick check, count the number of lines in each of the read files and check they have the same number. As these are paired end reads, there should be one read from each read pair in each of the files - and hence the same number of lines (and therefore reads) in each file. 

In [ ]:
%cd Module_11
!ls

In [ ]:
!gzip -cdf ERR2667737_1.fastq.gz | wc -l

In [ ]:
!gzip -cdf ERR2667737_2.fastq.gz | wc -l

An explanation of these commands is as follows:

`gzip`: Command to compress or uncompress files

`cdf`: Options where c means writing file on standard output, keeping the original files unchanged; d means decompress (to .fastq); and f means forcing overwrite of output files and use of symbolic links. (Here, fastq.gz files are symbolic links)

`|`: Pipes the output .fastq file to the next command

`wc`:  command for a program called word count, by providing the `-l` flag we tell word count to count the number of lines in a file.

Paired read files should always have the same number of lines/reads (the ordering of the reads in each file is also critical), so if your two paired files have a different number of lines, something has gone wrong (e.g., filtering/trimming went wrong and corrupted the output, or maybe files from different samples are being use

## Part 2: Creating an alignment

### Step 1: Reference genome indexing

We will first create an index of the reference sequence file “Reference_sequence_GPSC46.fa”. For large references, this can take a while but once the index is created you can re-use it for multiple samples. For example, if you have 100 Streptococcus pneumoniae samples, you only need to create the index once and you can use it for all 100 samples. 

Run the command in terminal to execute bwa:

In [ ]:
# Reference genome indexing
!bwa index Reference_sequence_GPSC1.fa

An explanation of this command is as follows:

`bwa`:  is the tool

`index`:  instructs bwa to index the input file

`Reference_sequence_GPSC1.fa`: is the input file

If you list  `ls` he content of the directory, you should now see the bwa index files, they will have the prefix “Reference_sequence_GPSC1” and will have extensions such as .amb, .ann, .bwt, .pac, and .sa.

In [ ]:
# List files in the current directory
!ls

### Step 2: Aligment

To align the reads to the reference sequence, type this command:

>Note: This process may take a long time

In [ ]:
# Align reads to the reference genome
!bwa mem Reference_sequence_GPSC1.fa ERR2667737_1.fastq.gz ERR2667737_2.fastq.gz > GPSC1bwa.sam

An explanation of this command is as follows:

`bwa`: is the tool

`mem`: tells bwa to use the mem algorithm to align the read files to the reference 

`ERR2667737_1.fastq.gz`: input file of forward reads

`ERR2667737_1.fastq.gz`: input file of reverse reads

`Reference_sequence_GPSC46.fa`: is the input file

`> GPSC1bwa.sam`: direct `>` the alignment results into the file GPSC46bwa.sam

When bwa runs, it will print messages to your terminal screen:

When bwa is finished, check that the SAM file has been created using `ls`

There should now be a file "GPSC1bwa.sam" in the directory. Typically, a SAM file contains a single line for each read in the data set, and this line stores the alignment results of each read (reference name, alignment location, CIGAR string, the read sequence itself, quality etc..)

SAM files are in text format. To view it, type: 

In [ ]:
# List files in the current directory

In [ ]:
# View the first few lines of the SAM file
!head GPSC1bwa.sam

You will have the following output:

![bwa](images/bwa.png)

### Step 3: Convert SAM to BAM

It is good practice to convert your SAM files to BAM files (Binary Alignment Map), which are compressed binary versions of the same data, and can be sorted and indexed easily to make searches faster. We will use samtools to convert our SAM to BAM, and sort and index the BAM file. 

To convert a sam file to a bam file, type this command in terminal:

In [ ]:
# Convert SAM to BAM
!samtools sort GPSC1bwa.sam -o GPSC1bwa.bam

An explanation of this command is as follows:

`samtools`: is the tool

`sort`: tells samtools to sort the SAM file (GPSC1bwa.sam)

`GPSC1bwa.sam`:  input file

`-o GPSC1bwa.bam`: flag for the output file called GPSC46bwa.bam which is the sorted data in the BAM format 

### Step 4: BAM file indexing

Indexing, which relies on sorted data, enables faster searches downstream. Type this command in terminal:

In [ ]:
# BAM file indexing
!samtools index GPSC1bwa.bam

An explanation of this command is as follows:

`samtools`:  is the tool

`index`: instructs samtools to index the input file (GPSC1bwa.bam)

`GPSC1bwa.bam`: input file

There should now be two new files called: GPSC46bwa.bam and GPSC46bwa.bam.bai (the BAM index file) in the directory. Now lets list (ls -alh) the content of the directory to check we have our new files, and also check out their sizes.


>Note: If your SAM file is 0B (i.e., 0 bytes) then something went wrong with the bwa alignment step, so restart from there. If your SAM file is fine (i.e., >0), but your BAM file is 0B (i.e., empty), then something went wrong with your SAM to BAM conversion so re-do that section.

We do not need our original SAM file anymore (as we have the BAM file now). So we remove `rm` the SAM file (GPSC1bwa.sam). 

### Step 5: Assessing the alignment

Samtools is one of the key pieces of software in analyses involving High Throughput Sequencing (HTS) data. It has a wide range of functions and combines easily with related tools such as vcftools (for calling variants). One common thing to check is how many reads have aligned to the reference (mapped), and how many did not (unmapped). Samtools can report this for us easily, utilising the aligner SAM flags you learnt in section 2 (NGS file formats).  

The 2nd column in the SAM file contains the flag for the read alignment. If the flag includes the number 4 flag in its makeup then the read is unmapped, if it doesn't include the number 4 then it is mapped.

**Number of mapped read alignments**

In [ ]:
# View the first few lines of the BAM file
!samtools view -c -F4 GPSC1bwa.bam

An explanation of this command is as follows:

`samtools view`: to view the file bwa.bam

`-c`: count the read alignment

`-F4`: skip read alignment that have the unmapped Flag 4

**Number of unmapped reads**

This time we use -f4, which only include read alignment that do have the unmapped flag 4

In [ ]:
# View the first few lines of the BAM file
!samtools view -c -f4 GPSC1bwa.bam

Technically, the above command gives the number of mapped read alignment not reads. A read could be mapped equally well to multiple positions (one will be called the primary alignment, and others secondary alignment [sam flag 256], or a read could be split into two parts (e.g., splicing) with one part being the primary alignment and the others supplementary [sam flag 2048].

So to get the true number of mapped reads you need to count the alignment that do not have flags 4 (unmapped), 256 (not primary), and 2048 (supplementary) = 4+256+2048 = 2308

**Number of mapped read reads**

In [ ]:
!samtools view -c -F2308 GPSC1bwa.bam

In [ ]:
!samtools view -c -F4 -F256 -F2048 GPSC1bwa.bam

**Statistics**

You can generate the statistics of mapping using the command

In [ ]:
!samtools stats  GPSC1bwa.bam > GPSC1bwa_bamstats.txt

Open the "GPSC1bwa_bamstats.txt" file to view the statistics for all the reads

## Par 3: View mapped reads in IGV 

Download [IGV](https://igv.org/doc/desktop/#DownloadPage/) 

You will require the following files to view mapped reads in IGV: 

- Reference sequence that you used to map your reads

- Sorted and indexed mapped read BAM file

Launch IGV 

1. Load the reference sequence: In the toolbar, Click Genome > Load Genome from file > Search and select Reference_sequence_GPSC1.fa

2. Load the BAM file: Go to File > Load from file > Select GPSC1bwa.bam

And you are now free to investigate different areas and the alignments in the genome.

## Part 4: Now perform a new alignment following the previous steps using `Reference_sequence_GPSC33` as the reference genome.

> Note: Essentially, you will need to index the new reference genome, change the names to the new reference genome in the `bwa` command, and update all SAM/BAM file names in the `bwa` and `samtools` commands from GPSC1 to GPSC33. Example:

`bwa mem Reference_sequence_GPSC33.fa ERR2667737_1.fastq.gz ERR2667737_2.fastq.gz > GPSC33bwa.sam`.


*Adapted from:*

- Advanced Bioinformatics Course developed for the GPS and JUNO projects - Wellcome Sanger Insitute

*Modified by Luisa Sacristán (Universidad de los Andes-CABANA)*